# 5.1 학습 모델 저장하고 재사용하기

```
0, 0, 1, 0, 0
1, 0, 0, 1, 0
1, 1, 0, 0, 1
0, 0, 1, 0, 0
0, 0, 1, 0, 0
0, 1, 0, 0, 1
```

위와 같은 내용을 data.csv파일로 저장
털, 날개, 기타, 포유류, 조류 순.

다음 csv를 np를 이용해 읽고 데이터를 가공한다.
여기서 unpack=True와 np.transpose는 데이터의 행과 열을 바꾼다는 의미 

In [1]:
# 모델을 저장하고 재사용하는 방법을 익혀봅니다.

import tensorflow as tf
import numpy as np


data = np.loadtxt('./data.csv', delimiter=',',unpack=True, dtype='float32')

# 털, 날개, 기타, 포유류, 조류
# x_data = 0, 1
# y_data = 2, 3, 4
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

ValueError: could not convert string to float: b' 0 1'

모델을 저장할 변수를 하나 만들고 (직접 쓰이진 않으므로 trainable을 False로 둠) 
다음 3단계의 신경망 구성을 만든다.

두번째 가중치에서 20으로 뉴런수를 늘리면 복잡도가 높은 문제를 해결하는데 도움을 주지만 과적합이라는 문제에 빠짐 (이 부분은 영근이가 설명)

최적화 후에는 global_step을 1씩 증가시켜 줌 

In [18]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# global_step로 넘겨준 변수를, 학습용 변수들을 최적화 할 때 마다 학습 횟수를 하나씩 증가시킵니다.
train_op = optimizer.minimize(cost, global_step=global_step)

이제 학습을 시켜야한다.
세션을 가져오고, 앞서 정의한 변수들을 가져온다 

In [19]:
sess = tf.Session()
# 모델을 저장하고 불러오는 API를 초기화합니다.
# global_variables 함수를 통해 앞서 정의하였던 변수들을 저장하거나 불러올 변수들로 설정합니다.
saver = tf.train.Saver(tf.global_variables())

# 기존에 학습한 모델이 있는지 ./model 디렉토리를 검색. 있으면 불러오고 없으면 초기화 
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 최적화 진행.
# 이전과는 달리 global_step을 통해서 학습이 몇 번째 진행하고 있는지를 출력해줌.
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 지정한 체크포인트 파일에 변수를 저장합니다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)




INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-14


NotFoundError: Key global_step_1 not found in checkpoint
	 [[Node: save_6/RestoreV2_23 = RestoreV2[dtypes=[DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_6/Const_0_0, save_6/RestoreV2_23/tensor_names, save_6/RestoreV2_23/shape_and_slices)]]

Caused by op 'save_6/RestoreV2_23', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-pa

In [ ]:
다음 결과확인을 위한 코드를 넣고 확인해봅니다.

In [20]:

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


FailedPreconditionError: Attempting to use uninitialized value Variable_5
	 [[Node: Variable_5/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_5"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_5)]]

Caused by op 'Variable_5/read', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
   

In [ ]:
# 텐서보드를 이용하기 위해 각종 변수들을 설정하고 저장하는 방법을 익혀봅니다.

import tensorflow as tf
import numpy as np


data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope 으로 묶은 블럭은 텐서보드에서 한 레이어안에 표현해줍니다
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    # tf.summary.scalar 를 이용해 수집하고 싶은 값들을 지정할 수 있습니다.
    tf.summary.scalar('cost', cost)


#########
# 신경망 모델 학습
######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 텐서보드에서 표시해주기 위한 텐서들을 수집합니다.
merged = tf.summary.merge_all()
# 저장할 그래프와 텐서값들을 저장할 디렉토리를 설정합니다.
writer = tf.summary.FileWriter('./logs', sess.graph)
# 이렇게 저장한 로그는, 학습 후 다음의 명령어를 이용해 웹서버를 실행시킨 뒤
# tensorboard --logdir=./logs
# 다음 주소와 웹브라우저를 이용해 텐서보드에서 확인할 수 있습니다.
# http://localhost:6006

# 최적화 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 적절한 시점에 저장할 값들을 수집하고 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


# 5.2 텐서보드 사용하기
딥러닝을 현업에 활용하려면 학습이 오래걸리기 때문에 학습과정을 추적하는 일이 중요. 텐서보드는 중간중간 손실값이나 정확도를 다양한 방식으로 시각화해서 보여줌.

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 앞의 코드에 비해서 이 부분이 추가 됨
with tf.name_scope('layer1'): # 계층을 묶어줌 
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

    tf.summary.histogram("X", X) # 각 가중치와 편향등의 변화 
    tf.summary.histogram("Weights", W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

    tf.summary.histogram("Weights", W2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

    tf.summary.histogram("Weights", W3)
    tf.summary.histogram("Model", model)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    tf.summary.scalar('cost', cost) # 손실값을 추적하기 위해 수집할 값을 지정하는 코드 scalar는 함수의 값이 하나일때 사용.
    # 더 자세한건 공식 문서 참조 

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 앞서 지정한 텐서들을 수집한 다음 filewriter함수를 이용해 그래프와 텐서의 값을 저장할 디렉터리를 생성 
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    # 모아둔 텐서의 값들을 계산하여 수집 
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    # 디렉터리에 저장. 나중에 확인할 수 있도록 global_step에 값 기록 
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Step: 1,  Cost: 1.206
Step: 2,  Cost: 1.115
Step: 3,  Cost: 1.033
Step: 4,  Cost: 0.968
Step: 5,  Cost: 0.912
Step: 6,  Cost: 0.864
Step: 7,  Cost: 0.824
Step: 8,  Cost: 0.791
Step: 9,  Cost: 0.762
Step: 10,  Cost: 0.737
Step: 11,  Cost: 0.715
Step: 12,  Cost: 0.697
Step: 13,  Cost: 0.683
Step: 14,  Cost: 0.669
Step: 15,  Cost: 0.658
Step: 16,  Cost: 0.647
Step: 17,  Cost: 0.638
Step: 18,  Cost: 0.630
Step: 19,  Cost: 0.623
Step: 20,  Cost: 0.616
Step: 21,  Cost: 0.610
Step: 22,  Cost: 0.605
Step: 23,  Cost: 0.600
Step: 24,  Cost: 0.596
Step: 25,  Cost: 0.592
Step: 26,  Cost: 0.589
Step: 27,  Cost: 0.585
Step: 28,  Cost: 0.583
Step: 29,  Cost: 0.580
Step: 30,  Cost: 0.578
Step: 31,  Cost: 0.575
Step: 32,  Cost: 0.573
Step: 33,  Cost: 0.571
Step: 34,  Cost: 0.570
Step: 35,  Cost: 0.568
Step: 36,  Cost: 0.567
Step: 37,  Cost: 0.565
Step: 38,  Cost: 0.564
Step: 39,  Cost: 0.563
Step: 40,  Cost: 0.562
Step: 41,  Cost: 0.561
Step: 42,  Cost: 0.560
Step: 43,  Cost: 0.559
Step: 44,  Cost: 0.5

실행시킬때 모델값이 바뀌었으므로 ./model을 지우고 시작 
시작하면 ./log에 데이터가 생김
터미널에 tensorboard --logdir=./logs 입력
하고 웹서버의 포트로 접속 